In [88]:
from typing import Dict, List

from solara.lab import Reactive
import reacton.ipyvuetify as v
import solara as sl
import openai

class Question:
    def __init__(self, q: str, a: str) -> None:
        self.q = q
        self.a = a

class State:
    question = Reactive[str]("")
    answer = Reactive[str]("<br><br><br>")
    history = Reactive[List[Question]]([])
    ans_btn_disabled = Reactive[bool](True)
    openai_key = Reactive[str]("")
    
    
def make_request(q: str) -> str:
    openai.api_key=State.openai_key.value
    res = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": q},
        ]
    )
    return res.choices[0].message.content

def query_gpt() -> str:
    if not State.openai_key.value:
        return
    ans = make_request(State.question.value)
    State.answer.set(ans)
    # Enable the save button
    State.ans_btn_disabled.set(False)


def save_answer() -> str:
    q = Question(q=State.question.value, a=State.answer.value)
    history = State.history.value.copy()
    history.append(q)
    State.history.set(history)
    
    
    
def get_table_rows() -> str:
    table_rows = ""
    for question in State.history.value:
        table_rows += f"<tr>\n\t<td>{question.q}</td>\n\t<td>{question.a}</td>\n</tr>"
    return table_rows
    
@sl.component
def qa() -> None:
    css = """
        .mybutton {
            left: 46%;
        }
    """
    with sl.Card():
        sl.Markdown('<h1 style="text-align: center;">Ask GPT</h1>')
        sl.InputText(
            "Ask your question", 
            value="", 
            on_value=State.question.set,
            continuous_update=True,
        )
        # Cant hit the button without adding an openai key
        need_tooltip = not State.openai_key.value
        if need_tooltip:
            with sl.Tooltip("Add OpenAI API Key to ask a question"):
                sl.Button("Get Answer", on_click=query_gpt, classes=["mybutton"])
        else:
            sl.Button("Get Answer", on_click=query_gpt, classes=["mybutton"])
        with sl.Card():
            sl.Markdown(State.answer.value)
        sl.Button(
            "Save Answer", 
            on_click=save_answer, 
            disabled=State.ans_btn_disabled.value,
            classes=["mybutton"]
        )
        sl.Style(css)
        

        
@sl.component
def history() -> None:
    with sl.Card():
        sl.Markdown('<h1 style="text-align: center;">Saved Q&A</h1><br>')
        sl.Markdown("---")
        css = """.center {
          margin-left: auto;
          margin-right: auto;
        }
        """
        print(get_table_rows())
        html = (f"""
            <table class='center' height="50%" width="100%" align=center cellpadding ="25">
              <tr>
                <th><h2>Question</h2></th>
                <th><h2>Answer</h2></th>
              </tr>
              {get_table_rows()}
            </table>
            """
        )
        
        sl.HTML(
            tag="div", unsafe_innerHTML=html, classes=["center"])
        sl.Markdown("---")
        sl.Style(css)
        

@sl.component
def sidebar() -> None:
    sl.InputText("OpenAI API Key", password=True, on_value=State.openai_key.set)


@sl.component
def Page() -> None:
    sl.Title("Ask GPT")
    with sl.Column():
        qa()
        history()
    with sl.Sidebar():
        sidebar()
        
        
        
    
Page()

Overwriting app.py
